In [119]:
import pandas as pd
import numpy as np
from sklearn.metrics .pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

vi_data_df = pd.read_csv ("vi_text_retrieval.csv")
context = vi_data_df ["text"]
context = [ doc . lower () for doc in context ]
print(context[1])
print(type(context))

một trận đấu bóng đá thông thường có hai hiệp , mỗi hiệp 45 phút với khoảng thời gian 15 phút nghỉ giữa hai hiệp .
<class 'list'>


In [120]:
tfidf_vectorizer = TfidfVectorizer ()
context_embedded = tfidf_vectorizer.fit_transform(context)
context_embedded.toarray()[7]

array([0.31126581, 0.        , 0.        , ..., 0.        , 0.        ,
       0.        ])

In [121]:
context_embedded = context_embedded.toarray()
print(context_embedded.shape)

(500, 2913)


In [122]:
question = "Quang Hải giành được chức vô địch U21 quốc gia năm bao nhiêu tuổi"
queery = [doc.lower() for doc in [question]]
querry_embedded = tfidf_vectorizer.transform(queery)

querry_embedded = querry_embedded.toarray().reshape(-1)
print(querry_embedded.shape)

(2913,)


In [123]:
dot = np.dot(context_embedded,querry_embedded)
#print(dot.argsort())
print(dot[0])

print(dot.shape)

# print(dot[7])
# print(dot)
print(np.linalg.norm(querry_embedded))
print(np.linalg.norm(context_embedded[0]))

0.6279910475266973
(500,)
1.0
1.0


## Cau 11

In [144]:
def cosine(query_embedded, database_embeded):
    numurator = np.dot(database_embeded,query_embedded)
    return cosine_similarity(query_embedded.reshape(1,-1), database_embeded)

# lowercasing before encoding
query_embedded = [doc.lower() for doc in [question]]
query_embedded = tfidf_vectorizer.transform(query_embedded)
query_embeded = query_embedded.toarray().reshape(-1)
print(query_embeded.shape)
print(query_embedded.reshape(1,-1).shape)
cosine_scores = cosine(query_embeded,context_embedded)

# print(cosine_scores)
print(cosine_scores[0][7])

(2913,)
(1, 2913)
0.03526487323867346


In [147]:
def tfidf_search (question , tfidf_vectorizer : TfidfVectorizer,context_embedded, top_d =5) :
    # lowercasing before encoding
    query_embedded = [doc.lower() for doc in [question]]
    query_embedded = tfidf_vectorizer.transform(query_embedded)
    query_embeded = query_embedded.toarray().reshape(-1)
    cosine_scores = cosine(query_embeded,context_embedded)[0]

    # Get top k cosine score and index its
    results = []
    for idx in cosine_scores.argsort()[-top_d:][:: -1]:
        doc_score = {
            "id": idx ,
            "cosine_score": cosine_scores[idx], 
            "sentence" :context[idx]
        }
        results.append(doc_score )
    return results

In [148]:
question = vi_data_df.iloc[0]["question"]
print(question)
results = tfidf_search (question, tfidf_vectorizer, context_embedded ,top_d =5)
results[0]["cosine_score"]

Quang Hải giành được chức vô địch U21 quốc gia năm bao nhiêu tuổi


0.6279910475266974

## Cau 12

In [174]:
def corrcoef(query_embedded, database_embeded):
    return np.corrcoef(query_embedded.reshape(1,-1), database_embeded)

def calculate_correlations_broadcast(vector, matrix):
    # Step 1: Center the vector and each row of the matrix by subtracting their means
    vector_mean = np.mean(vector)
    matrix_mean = np.mean(matrix, axis=1)
    
    vector_centered = vector - vector_mean
    matrix_centered = matrix - matrix_mean[:, None]
    
    # Step 2: Compute the dot product of the centered vector and each row of the centered matrix
    dot_products = np.dot(matrix_centered, vector_centered)
    
    # Step 3: Compute the norms of the centered vector and each row of the centered matrix
    vector_norm = np.linalg.norm(vector_centered)
    matrix_norms = np.linalg.norm(matrix_centered, axis=1)
    
    # Step 4: Divide the dot products by the product of the norms to get the correlation coefficients
    correlation_coefficients = dot_products / (vector_norm * matrix_norms)
    
    return correlation_coefficients

query_embedded = [doc.lower() for doc in [question]]
query_embedded = tfidf_vectorizer.transform(query_embedded)
query_embeded = query_embedded.toarray().reshape(-1)

print(query_embeded.shape)
print(context_embedded.shape)
corrcoef_scores = corrcoef(query_embeded,context_embedded)
print(corrcoef_scores[0][1:].shape)

(2913,)
(500, 2913)
(500,)


In [187]:
print(query_embeded.shape)
print(context_embedded.shape)
a = calculate_correlations_broadcast(query_embeded, context_embedded)
print(np.sort(a)[2:])

(2913,)
(500, 2913)
[-8.66079674e-03 -8.37829984e-03 -8.37829984e-03 -8.28306960e-03
 -8.14297643e-03 -8.03188324e-03 -7.66131275e-03 -7.64380001e-03
 -7.59848455e-03 -7.59848455e-03 -7.58105790e-03 -7.58105790e-03
 -7.56425037e-03 -7.55679466e-03 -7.34904651e-03 -7.33074485e-03
 -7.18442338e-03 -7.14970921e-03 -7.12948367e-03 -7.09342762e-03
 -6.96981610e-03 -6.96933712e-03 -6.95020428e-03 -6.95020428e-03
 -6.94880414e-03 -6.94467422e-03 -6.93753510e-03 -6.93753510e-03
 -6.92872330e-03 -6.92621421e-03 -6.86004894e-03 -6.81527713e-03
 -6.80236028e-03 -6.79990774e-03 -6.78129307e-03 -6.77786306e-03
 -6.71957676e-03 -6.70959558e-03 -6.70566986e-03 -6.69017813e-03
 -6.63805639e-03 -6.60862822e-03 -6.56930604e-03 -6.56375562e-03
 -6.56375562e-03 -6.54395342e-03 -6.40730772e-03 -6.37313167e-03
 -6.20928443e-03 -6.13087110e-03 -6.13087110e-03 -6.07526195e-03
 -6.06097349e-03 -6.03316833e-03 -5.97141341e-03 -5.97141341e-03
 -5.95736121e-03 -5.92842003e-03 -5.86049387e-03 -5.85413562e-03
 -5.8

In [172]:
def corr_search ( question , tfidf_vectorizer , context_embedded,top_d =5) :
    # lowercasing before encoding
    query_embedded = [doc.lower() for doc in [question]]
    query_embedded = tfidf_vectorizer.transform(query_embedded)
    query_embeded = query_embedded.toarray().reshape(-1)

    corr_scores = corrcoef(query_embeded,context_embedded )
    corr_scores = corr_scores[0][1:]

    results = []
    for idx in corr_scores.argsort()[-top_d :][:: -1]:
        doc = {
        "id": idx ,
        "corr_score": corr_scores[idx]
        }
    results.append(doc)
    return results  

In [173]:
question = vi_data_df.iloc[0]["question"]
print(question)
results = corr_search(question , tfidf_vectorizer ,context_embedded, top_d =5)
results [0]["corr_score"]

Quang Hải giành được chức vô địch U21 quốc gia năm bao nhiêu tuổi


0.15444531996663344